In [ ]:
import os
import datetime
from glob import glob
import IPython.display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.optimize
import scipy.interpolate

import pydicom

In [ ]:
data_root = r'S:\Physics\Programming\data\MVISO\NBCC\4299\20190415'

In [ ]:
output_csv = os.path.join(data_root, 'results.csv')

In [ ]:
data_record = glob(os.path.join(data_root, 'iView*.xlsx'))[0]
dicom_files = np.array(glob(os.path.join(data_root, '*.dcm')))

In [ ]:
record = pd.read_excel(data_record, skiprows=4)
timestamps_initial = record['Datetime']
timestamps = timestamps_initial[timestamps_initial.notnull()].values
gantry = record['Gantry'][timestamps_initial.notnull()].values
collimator = record['Col'][timestamps_initial.notnull()].values
turntable = record['TT'][timestamps_initial.notnull()].values
beam = record['Energy'][timestamps_initial.notnull()].values

In [ ]:
datasets = np.array([
    pydicom.read_file(dicom_file, force=True)
    for dicom_file in dicom_files
])

In [ ]:
acquisition_datetimes = np.array([
    datetime.datetime.strptime(dataset.AcquisitionDate + dataset.AcquisitionTime, '%Y%m%d%H%M%S.%f')
    for dataset in datasets
], dtype=np.datetime64)

In [ ]:
diff_map = np.abs(acquisition_datetimes[None,:] - timestamps[:, None]) < np.timedelta64(2, 's')
timestamp_index, acquisition_index = np.where(diff_map)

In [ ]:
assert len(set(acquisition_index)) == len(acquisition_index)
assert len(acquisition_index) == len(acquisition_datetimes)

In [ ]:
datasets = datasets[acquisition_index]
dicom_files = dicom_files[acquisition_index]
timestamps = timestamps[timestamp_index]
gantry = gantry[timestamp_index]
collimator = collimator[timestamp_index]
turntable = turntable[timestamp_index]
beam = beam[timestamp_index]

acquisition_datetimes = np.array([
    datetime.datetime.strptime(dataset.AcquisitionDate + dataset.AcquisitionTime, '%Y%m%d%H%M%S.%f')
    for dataset in datasets
], dtype=np.datetime64)

diff_map = np.abs(acquisition_datetimes[None,:] - timestamps[:, None]) < np.timedelta64(2, 's')
timestamp_index, acquisition_index = np.where(diff_map)

assert np.all(timestamp_index == acquisition_index)

In [ ]:
flipped_pixel_arrays = np.array([
    dataset.pixel_array[::-1,:]
    for dataset in datasets
], copy=True)

flipped_pixel_arrays = 1 - flipped_pixel_arrays/2**16

In [ ]:
axis_distance = np.arange(-512, 512)/4

initial_mask_distance = 20  # mm

first = np.where(axis_distance >= -initial_mask_distance)[0][0]
last = np.where(axis_distance > initial_mask_distance)[0][0]

mask = slice(first, last)

axis_distance = axis_distance[mask]

masked_arrays = np.array([
    pixel_array[mask, mask]
    for pixel_array in flipped_pixel_arrays
])

In [ ]:
square_field_side_length = 20  # mm

penumbra_width = 3  # mm
ball_bearing_diameter = 8 # mm

In [ ]:
dx = 0.05
interpolated_distances = np.arange(-initial_mask_distance, initial_mask_distance+dx, dx)

xx, yy = np.meshgrid(interpolated_distances, interpolated_distances)
xx_flat = np.ravel(xx)
yy_flat = np.ravel(yy)

In [ ]:
import numpy as np
from scipy.special import erf, erfinv  # pylint: disable=no-name-in-module


def gaussian_cdf(x, mu=0, sig=1):
    x = np.array(x, copy=False)
    return 0.5 * (1 + erf((x - mu) / (sig * np.sqrt(2))))


def scaled_penumbra_sig(profile_shoulder_edge=0.8):
    sig = 1 / (2 * np.sqrt(2) * erfinv(profile_shoulder_edge * 2 - 1))

    return sig


def create_dummy_profile_function(centre, field_width, penumbra_width):
    sig = scaled_penumbra_sig() * penumbra_width
    mu = [centre - field_width/2, centre + field_width/2]

    def dummy_profile(x):
        return gaussian_cdf(x, mu[0], sig) * gaussian_cdf(-x, -mu[1], sig)

    return dummy_profile

In [ ]:
x = np.linspace(-10, 10, 1000)

dummy_profile = create_dummy_profile_function(1, 12, 0.5)

plt.plot(x, dummy_profile(x))

In [ ]:
from pymedphys_analysis.mocks.profiles import create_dummy_profile_function
from pymedphys_analysis.winstonlutz.profiles import penumbra_flip_diff


profile_centre = 1.7
field_width = 10
penumbra_width = 0.3

dummy_profile = create_dummy_profile_function(
    profile_centre, field_width, penumbra_width)

x = np.linspace(-10, 10, 100)
y = dummy_profile(x)

np.round(y, 2)

In [ ]:
type(dum)

In [ ]:
import scipy.special

x = np.linspace(-2.5,2.5)
mu = 0
sig = 1


def gaussian_cdf(x, mu=0, sig=1):
    return 0.5 * (1 + scipy.special.erf((x - mu) / (sig * np.sqrt(2))))


plt.plot(x, gauss_cdf(x, mu, sig))


In [ ]:
def scaled_penumbra_sig(penumbra_shoulder=0.8):
    sig = 2 * np.sqrt(2) * scipy.special.erfinv(penumbra_shoulder * 2 - 1)
    
    return sig

In [ ]:
def create_dummy_profile_function(centre, field_width, penumbra_width):  
    sig = scaled_penumbra_sig()
    
    
    
    

In [ ]:
sig = scaled_penumbra_sig(0.8)
sig

In [ ]:
gaussian_cdf(0.5, sig=sig)

In [ ]:
def edge_normalisation()

In [ ]:
x = np.linspace(-10, 10, 1000)

penumb_bound = (sig*np.sqrt(2) * scipy.special.erfinv(0.6) + mu) * 2

plt.plot(x, gauss_cdf(x, mu-2, sig/penumb_bound))

In [ ]:
x = np.linspace(-2.5,2.5, 1000)
y = gauss_cdf(x, mu, sig)

i = 332
j = 668

x[332]

In [ ]:
gauss_cdf(0, mu, sig/penumb_bound)

In [ ]:
gauss_cdf(-0.84165, mu, sig)

In [ ]:
x[668]

In [ ]:

import os
import datetime
from glob import glob
import IPython.display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.optimize
import scipy.interpolate

import pydicom

from pymedphys_analysis.mocks.profiles import create_square_field_function

In [ ]:
field = create_square_field_function([5,5], 2, 0.3, 10)

In [ ]:
    xx, yy = np.meshgrid(
        np.linspace(3, 7, 10),
        [4, 5, 6]
    )

In [ ]:
np.round(field(xx, yy), 2)

In [ ]:
np.round(field(np.linspace(3,7,10), [5] * 10), 2)

In [ ]:
np.round(field(np.linspace(3,7,10), [4] * 10), 2)

In [ ]:
np.round(field(np.linspace(3,7,10), [6] * 10), 2)

In [ ]:
x = np.linspace(3, 7)
xx, yy = np.meshgrid(x, x)

zz = field(xx, yy)

In [ ]:
plt.contour(xx, yy, zz, 10)
plt.axis('equal')
plt.grid()

In [ ]:
x = np.linspace(-10,10)
y = np

In [ ]:
penumbra_range = np.linspace(-penumbra_width/2, penumbra_width/2, 11)
half_field_range = np.linspace(-square_field_side_length/4, square_field_side_length/4, 51)

def get_sum_of_square_penumbra_flip(centre_x, centre_y, interpolation_func):
    left_lookup = centre_x - square_field_side_length/2 + penumbra_range
    right_lookup = centre_x + square_field_side_length/2 + penumbra_range
    x_lookup = np.concatenate([left_lookup, right_lookup])

    y_lookup = centre_y + half_field_range

    xx_lookup, yy_lookup = np.meshgrid(x_lookup, y_lookup)
    xx_lookup_flat = np.ravel(xx_lookup)
    yy_lookup_flat = np.ravel(yy_lookup)

    penumbra_values_flat = interpolation_func(yy_lookup_flat, xx_lookup_flat)
    penumbra_values = np.reshape(penumbra_values_flat, np.shape(xx_lookup))
    
    weighted_diff = 2*(penumbra_values - penumbra_values[:, ::-1]) / (penumbra_values + penumbra_values[:, ::-1])
    
    return np.sum((weighted_diff)**2)
    
    
def get_sum_of_square_penumbra_flip_transpose(centre_x, centre_y, interpolation_func):
    def transposed_interp_func(y, x):
        return interpolation_func(x, y)
        
    return get_sum_of_square_penumbra_flip(centre_y, centre_x, transposed_interp_func)


def get_sum_of_square_both_penumbra_flips(centre_x, centre_y, interpolation):
    interpolation_func = interpolation.ev
    
    return (
        get_sum_of_square_penumbra_flip(centre_x, centre_y, interpolation_func) + 
        get_sum_of_square_penumbra_flip_transpose(centre_x, centre_y, interpolation_func)
    )


def create_penumbra_minimisation(interpolation):
    def to_minimise(centre):
        return get_sum_of_square_both_penumbra_flips(centre[1], centre[0], interpolation)
    
    return to_minimise

In [ ]:
def show_image(pixel_array):
    plt.pcolormesh(interpolated_distances, interpolated_distances, pixel_array, clim=[0, 1])
    plt.colorbar()
    plt.axis('equal')

In [ ]:
def create_print_func(image_to_search):
    def print_fun(centre, f, accepted):
        print(centre)
        print(f)
        print(accepted)
        
        show_image_with_square(image_to_search, centre, square_field_side_length)
        
    return print_fun

In [ ]:
def create_points_to_check():
    number_in_circle = 31
    dtheta = 2*np.pi / number_in_circle
    t = np.arange(0, 2*np.pi, dtheta)
    diameters = ball_bearing_diameter * np.arange(0.1, 0.5, 0.05)

    def points_to_check(bb_centre):
        x = []
        y = []
        weight = []
        for i, diameter in enumerate(diameters):        
            x.append(diameter/2 * np.sin(t + i*dtheta/3) + bb_centre[1])
            y.append(diameter/2 * np.cos(t + i*dtheta/3) + bb_centre[0])
            
            weight.append(np.repeat(np.cos(np.arcsin(diameter/ball_bearing_diameter)), number_in_circle))
            
        x = np.vstack(x)
        y = np.vstack(y)
        weight = np.vstack(weight)
        
        return x, y, weight
    
    return points_to_check

    
points_to_check = create_points_to_check()
x, y, weight = points_to_check([0,0])


plt.plot(x.T, y.T, '.')
plt.axis('equal')

In [ ]:
def check_points_by_value(bb_centre, field_centre, interpolation):
    x, y, weight = points_to_check(bb_centre)
    
    results = weight * interpolation.ev(y, x)
    results_cost = 10 * np.mean(results)

    return results_cost


def check_points_by_symmetry(bb_centre, field_centre, interpolation):
    x, y, weight = points_to_check(bb_centre)
    
    results = weight * interpolation.ev(y, x)  
    results_mean = np.mean(results, axis=1)
    
    symmetry = np.mean(((results - results_mean[:,None])/results_mean[:,None])**2)
    gradient = np.sum(-np.diff(results_mean) / results_mean[1::])
    
    return symmetry + gradient

def create_circle_to_minimise_stage_1(field_centre, interpolation):
    def circle_to_minimise(bb_centre):
        return check_points_by_value(bb_centre, field_centre, interpolation)
    
    return circle_to_minimise


def create_circle_to_minimise_stage_2(field_centre, interpolation):
    def circle_to_minimise(bb_centre):
        return check_points_by_symmetry(bb_centre, field_centre, interpolation)
    
    return circle_to_minimise

In [ ]:
field_axis_bound = initial_mask_distance - square_field_side_length/2

field_bounds = [
    (-field_axis_bound, field_axis_bound), (-field_axis_bound, field_axis_bound)
]

In [ ]:
def find_ballbearing_centre(field_centre, interpolation):
    circle_bounds = [
        (
            field_centre[0] - square_field_side_length*0.8 / 2 + ball_bearing_diameter/2,
            field_centre[0] + square_field_side_length*0.8 / 2 - ball_bearing_diameter/2
        ),
        (
            field_centre[1] - square_field_side_length*0.8 / 2 + ball_bearing_diameter/2,
            field_centre[1] + square_field_side_length*0.8 / 2 - ball_bearing_diameter/2
        ),
    ]
    
    circle_to_minimise_stage_1 = create_circle_to_minimise_stage_1(field_centre, interpolation)
    
    bb_results = scipy.optimize.basinhopping(
        circle_to_minimise_stage_1, field_centre, T=1, niter=200, niter_success=5, stepsize=0.25,
        minimizer_kwargs={
            'method': 'L-BFGS-B',
            'bounds': circle_bounds
        }
    )
    bb_centre = bb_results.x
    
#     circle_to_minimise_stage_2 = create_circle_to_minimise_stage_2(field_centre, interpolation)
    
#     bb_results = scipy.optimize.minimize(
#         circle_to_minimise_stage_2, bb_centre, method='L-BFGS-B', bounds=circle_bounds)
#     bb_centre = bb_results.x
    
    
    if np.any(np.array(bb_centre)[:,None,None] == np.array(circle_bounds)[None,:,:]):
        print("BB found at bounds, retrying...")
        bb_centre = find_ballbearing_centre(field_centre, interpolation)
    
    return bb_centre
    

In [ ]:
interpolations = []
field_centres = []
bb_centres = []
field_displacements = []

for i, masked_array in enumerate(masked_arrays):  
    interpolation = scipy.interpolate.RectBivariateSpline(axis_distance, axis_distance, masked_array, kx=1, ky=1)
    interpolations.append(interpolation)
    
    field_to_minimise = create_penumbra_minimisation(interpolation)
    
    field_results = scipy.optimize.basinhopping(
        field_to_minimise, [0,0], T=1, niter=200, niter_success=5, stepsize=0.25, 
        minimizer_kwargs={
            'method': 'L-BFGS-B',
            'bounds': field_bounds
        }
    )
    field_centre = field_results.x
    
    field_centres.append(field_centre)
    
    bb_centre = find_ballbearing_centre(field_centre, interpolation)
    bb_centres.append(bb_centre)
    
    field_displacement = (field_centre[0] - bb_centre[0], field_centre[1] - bb_centre[1])
    field_displacements.append(field_displacement)

In [ ]:
field_centres = np.array(field_centres)
bb_centres = np.array(bb_centres)
field_displacements = np.array(field_displacements)

In [ ]:
def plot_square_and_circle_at_field(ax, bb_centre, field_centre, edge_length):    
    ax.plot(
        [
            field_centre[1] - edge_length/2, 
            field_centre[1] - edge_length/2, 
            field_centre[1] + edge_length/2,
            field_centre[1] + edge_length/2,
            field_centre[1] - edge_length/2],
        [
            field_centre[0] - edge_length/2, 
            field_centre[0] + edge_length/2, 
            field_centre[0] + edge_length/2,  
            field_centre[0] - edge_length/2, 
            field_centre[0] - edge_length/2],
        'k', lw=2
    )
    
    ax.plot([field_centre[1], bb_centre[1]], [field_centre[0], bb_centre[0]], 'r-', lw=2)

    ax.plot(
        [field_centre[1]-square_field_side_length/2, field_centre[1]+square_field_side_length/2],
        [field_centre[0]-square_field_side_length/2, field_centre[0]+square_field_side_length/2],
        'k', lw=0.5
    )

    ax.plot(
        [field_centre[1]-square_field_side_length/2, field_centre[1]+square_field_side_length/2],
        [field_centre[0]+square_field_side_length/2, field_centre[0]-square_field_side_length/2],
        'k', lw=0.5
    )

    ax.plot(
        [bb_centre[1]-ball_bearing_diameter/2, bb_centre[1]+ball_bearing_diameter/2],
        [bb_centre[0], bb_centre[0]],
        'k', lw=0.5
    )

    ax.plot(
        [bb_centre[1], bb_centre[1]],
        [bb_centre[0]+ball_bearing_diameter/2, bb_centre[0]-ball_bearing_diameter/2],
        'k', lw=0.5
    )
    
    t = np.linspace(0, 2*np.pi)
    circle_x = ball_bearing_diameter/2 * np.sin(t) + bb_centre[1]
    circle_y = ball_bearing_diameter/2 * np.cos(t) + bb_centre[0]
    
    ax.plot(circle_x, circle_y, 'k', lw=2)

In [ ]:
# i = 0

# (
#     interpolation, field_centre, bb_centre, field_displacement
# ) = interpolations[i], field_centres[i], bb_centres[i], field_displacements[i]


# def full_plot(interpolation, field_centre, bb_centre, field_displacement):
#     plt.figure(figsize=(10,10))

#     plot_circle_at_bb(bb_centre)

#     interpolated_image_flat = interpolation.ev(yy_flat, xx_flat)
#     interpolated_image = np.reshape(interpolated_image_flat, np.shape(xx))

#     show_image_with_square(interpolated_image, field_centre, square_field_side_length)
#     plt.plot([field_centre[1], bb_centre[1]], [field_centre[0], bb_centre[0]], 'r-', lw=2)

#     plt.plot(
#         [field_centre[1]-square_field_side_length/2, field_centre[1]+square_field_side_length/2],
#         [field_centre[0]-square_field_side_length/2, field_centre[0]+square_field_side_length/2],
#         'k', lw=0.5
#     )

#     plt.plot(
#         [field_centre[1]-square_field_side_length/2, field_centre[1]+square_field_side_length/2],
#         [field_centre[0]+square_field_side_length/2, field_centre[0]-square_field_side_length/2],
#         'k', lw=0.5
#     )

#     plt.plot(
#         [bb_centre[1]-ball_bearing_diameter/2, bb_centre[1]+ball_bearing_diameter/2],
#         [bb_centre[0], bb_centre[0]],
#         'k', lw=0.5
#     )

#     plt.plot(
#         [bb_centre[1], bb_centre[1]],
#         [bb_centre[0]+ball_bearing_diameter/2, bb_centre[0]-ball_bearing_diameter/2],
#         'k', lw=0.5
#     )


#     assert np.all(field_displacement == (field_centre[0] - bb_centre[0], field_centre[1] - bb_centre[1]))

#     title_text = (
#         "Beam: {} | Gantry: {:.0f} | Collimator: {:.0f} | TurnTable: {:.0f} | Displacement (x, y): ({:.2f}, {:.2f})".format(
#             beam[i], gantry[i], collimator[i], turntable[i], field_displacement[1], field_displacement[0])
#     )
#     plt.title(title_text)
    

# full_plot(interpolation, field_centre, bb_centre, field_displacement)

In [ ]:
tabulated_data = pd.DataFrame(
    index=timestamps,
    data=np.array([
        beam, gantry, collimator, turntable, 
        field_centres[:, 1], field_centres[:, 0],
        bb_centres[:, 1], bb_centres[:, 0],
        field_displacements[:, 1], field_displacements[:, 0]
    ]).T,
    columns=[
        'Beam', 'Gantry', 'Collimator', 'Turn Table', 
        'Field Centre x (mm)', 'Field Centre y (mm)',
        'BB Centre x (mm)', 'BB Centre y (mm)',
        'Field - BB x (mm)', 'Field - BB y (mm)',
    ]
)

In [ ]:
tabulated_data.to_csv(output_csv)

In [ ]:
string_timestamps = tabulated_data.index.strftime('%Y%m%d_%H%M%S').values

In [ ]:
def create_combined_inspection_image(masked_array, interpolation, field_centre, bb_centre, field_displacement):
    centres = np.hstack([field_centres, bb_centres])
    closest_image_indices = np.argmin(
        np.abs(axis_distance[:,None,None] - centres[None,:,:]), axis=0)
    
    closest_image_index = closest_image_indices[i]
    
    interpolated_image_flat = interpolation.ev(yy_flat, xx_flat)
    interpolated_image = np.reshape(interpolated_image_flat, np.shape(xx))

    fig, ((ax11, ax12), (ax21, ax22), (ax31, ax32)) = plt.subplots(3, 2, figsize=(12,15))

    title_text = (
        "Beam: {} | Gantry: {:.0f} | Collimator: {:.0f} | TurnTable: {:.0f} | Displacement (x, y): ({:.2f}, {:.2f})".format(
            beam[i], gantry[i], collimator[i], turntable[i], field_displacement[1], field_displacement[0])
    )
    fig.suptitle(title_text, fontsize=16)

    c11 = ax11.pcolormesh(axis_distance, axis_distance, masked_array, clim=[0, 1])
    fig.colorbar(c11, ax=ax11, label='Scaled image pixel value')
    ax11.axis('equal')
    ax11.set_title('Original image resolution')
    ax11.set_xlabel('x (mm)')
    ax11.set_ylabel('y (mm)')

    c12 = ax12.pcolormesh(interpolated_distances, interpolated_distances, interpolated_image, clim=[0, 1])
    fig.colorbar(c12, ax=ax12, label='Scaled image pixel value')

    plot_square_and_circle_at_field(ax12, bb_centre, field_centre, square_field_side_length)
    ax12.set_title('Interpolated image with field and BB overlay')
    ax12.axis('equal')
    ax12.set_xlabel('x (mm)')
    ax12.set_ylabel('y (mm)')

    ax21.set_title("x centred and flipped about field centre")
    ax21.plot(axis_distance - field_centre[1], masked_array[closest_image_index[0],:])
    ax21.plot(field_centre[1] - axis_distance, masked_array[closest_image_index[0],:])
    ax21.set_xlabel('x (mm)')
    ax21.set_ylabel('Scaled image pixel value')

    ax22.set_title("y centred and flipped about field centre")
    ax22.plot(axis_distance - field_centre[0], masked_array[:,closest_image_index[1]])
    ax22.plot(field_centre[0] - axis_distance, masked_array[:,closest_image_index[1]])
    ax22.set_xlabel('y (mm)')
    ax22.set_ylabel('Scaled image pixel value')

    ax31.set_title("x centred and flipped about ball bearing centre")
    ax31.plot(axis_distance - bb_centre[1], masked_array[closest_image_index[2],:])
    ax31.plot(bb_centre[1] - axis_distance, masked_array[closest_image_index[2],:])
    ax31.set_xlabel('x (mm)')
    ax31.set_ylabel('Scaled image pixel value')

    ax32.set_title("y centred and flipped about ball bearing centre")
    ax32.plot(axis_distance - bb_centre[0], masked_array[:,closest_image_index[3]])
    ax32.plot(bb_centre[0] - axis_distance, masked_array[:,closest_image_index[3]])
    ax32.set_xlabel('y (mm)')
    ax32.set_ylabel('Scaled image pixel value')
    

    
# i = 0

# # axis_distance
# masked_array = masked_arrays[i]
# bb_centre = bb_centres[i]
# field_centre = field_centres[i]
# interpolation = interpolations[i]
# field_displacement = field_displacements[i]

# create_combined_inspection_image(masked_array, interpolation, field_centre, bb_centre, field_displacement)

In [ ]:
# fff_6MV_ref = (tabulated_data['Beam'] == '6FFF') | (tabulated_data['Beam'] == '6fff')

# fff_6MV = tabulated_data[fff_6MV_ref]
# plt.scatter(fff_6MV['Gantry'], fff_6MV['Field - BB x (mm)'], label='x deviation')
# plt.scatter(fff_6MV['Gantry'], fff_6MV['Field - BB y (mm)'], label='y deviation')

# plt.xlabel('Gantry Angle')
# plt.ylabel('Field - BB iView Imaging Plane Deviation (mm)')

# plt.legend()
# plt.title('6MV FFF')

In [ ]:
# def plot_a_subset(tabulated_data, beam, colour_gen):
#     ref = tabulated_data['Beam'] == beam
#     subset = tabulated_data[ref]
#     sorted_subset = subset.sort_values('Gantry')
    
#     colour = next(colour_gen)
    
#     plt.plot(sorted_subset['Gantry'], sorted_subset['Field - BB x (mm)'], 'o--', label='{} (x)'.format(beam), c=colour)
# #     plt.plot(sorted_subset['Gantry'], sorted_subset['Field - BB y (mm)'], 's-.', label='{} (y)'.format(beam), c=colour)



In [ ]:
# def colours():
#     cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
#     for colour in cycle:
#         yield colour

In [ ]:
# colour_gen = colours()
# plt.figure(figsize=(15,10))

# plot_a_subset(tabulated_data, '6Flat', colour_gen)
# plot_a_subset(tabulated_data, '10Flat', colour_gen)
# plot_a_subset(tabulated_data, '6FFF', colour_gen)
# plot_a_subset(tabulated_data, '10FFF', colour_gen)

# plt.xlabel('Gantry Angle (degrees)')
# plt.ylabel('Field - BB iView Imaging Plane Deviation (mm)')

# plt.grid(which='major')
# plt.grid(which='minor', linestyle='--')
# plt.minorticks_on()

# plt.legend()
# plt.title('6MV Flat and 6MV FFF, varying gantry. Colimator 0, TT 0.')

In [ ]:
for i, (interpolation, field_centre, bb_centre) in enumerate(zip(interpolations, field_centres, bb_centres)):   

    create_combined_inspection_image(
        masked_arrays[i], interpolation, field_centre, bb_centre, field_displacements[i])
    file_name = "{}_{}_G{:+04.0f}_C{:+04.0f}_TT{:+03.0f}".format(
        string_timestamps[i], beam[i], gantry[i], collimator[i], turntable[i])
    
    file_path = os.path.join(data_root, file_name)
    
    plt.savefig(file_path)    

    plt.show()